In [1]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import img_to_array
from imutils.object_detection import non_max_suppression
import numpy as np
import argparse
import cv2

### Selective search는
- 전통적인 객체 검출에서는 image pyramid와 sliding windows를 통해서 이미지내 객체 검출을 했는데,
- 속도와 파라미터 설정에 따라 객체 검출 성능이 차이가 많아 문제가 됨
- 이를 해결하기 위해 selective search로 대체됨

In [2]:
def selective_search(image, method="fast"):
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(image)

    if method == "fast":
        ss.switchToSelectiveSearchFast()
    else:
        ss.switchToSelectiveSearchQuality()
    rects = ss.process()

    return rects

In [3]:
image = './beagle.png'
method = 'fast'  # or 'quality'
conf = 0.9
#labelFilters = ''
#labelFilters = labelFilters.lower().split(",")

In [4]:
print("[INFO] loading ResNet...")
model = ResNet50(weights="imagenet")

image = cv2.imread(image)
(H, W) = image.shape[:2]

[INFO] loading ResNet...


In [5]:
print("[INFO] performing selective search with '{}' method...".format(method))
rects = selective_search(image, method=method)
print("[INFO] {} regions found by selective search".format(len(rects)))

proposals = []
boxes = []

[INFO] performing selective search with 'fast' method...
[INFO] 922 regions found by selective search


In [6]:
for (x, y, w, h) in rects:
    # 오리지널 이미지에 w 또는 h가 10%이하 크기의 object는 무시
    if w / float(W) < 0.1 or h / float(H) < 0.1:
        continue
    # BGR형식을 RGB형식으로 변환 후
    # ResNet50의 basic 크기인 (224, 224)로 resize
    roi = image[y:y + h, x:x + w]
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    roi = cv2.resize(roi, (224, 224))
    # ROI 전처리
    roi = img_to_array(roi)
    roi = preprocess_input(roi)
    # update proposals and bounding boxes lists
    proposals.append(roi)
    boxes.append((x, y, w, h))

In [10]:
proposals[:1]

[array([[[ -40.939003  ,    3.2210007 ,   42.32      ],
         [ -40.939003  ,    3.2210007 ,   42.32      ],
         [ -41.939003  ,    2.2210007 ,   41.32      ],
         ...,
         [ -45.939003  ,   -2.7789993 ,   37.32      ],
         [ -32.939003  ,   10.221001  ,   50.32      ],
         [ -27.939003  ,   15.221001  ,   54.32      ]],
 
        [[ -40.939003  ,    3.2210007 ,   42.32      ],
         [ -40.939003  ,    3.2210007 ,   41.32      ],
         [ -41.939003  ,    2.2210007 ,   41.32      ],
         ...,
         [ -45.939003  ,   -2.7789993 ,   37.32      ],
         [ -32.939003  ,   11.221001  ,   50.32      ],
         [ -27.939003  ,   15.221001  ,   54.32      ]],
 
        [[ -42.939003  ,    2.2210007 ,   41.32      ],
         [ -41.939003  ,    2.2210007 ,   41.32      ],
         [ -42.939003  ,    2.2210007 ,   41.32      ],
         ...,
         [ -44.939003  ,   -1.7789993 ,   38.32      ],
         [ -31.939003  ,   12.221001  ,   50.32      ],


In [9]:
boxes[:3]

[(0, 0, 96, 74), (362, 0, 138, 109), (359, 0, 141, 336)]